Testing how the methods performs as we increase the rank of $A$

In [1]:
workspace()

In [4]:
using Plots
using JLD
include("../src/Pseudoinverse.jl")
importall Pseudoinverse

In [2]:
#Basic parameters
tol = 10.0^(-2); restol = 10.0^(-4);
maxiter=1000;
max_time = 10;
printiters = false;
exacterror =false;

In [ ]:
n = 50;
m= 10;
max_time = 5;
rs = 1:1:10;
methodstot = 3;
prob = uniform_mat_rank(m,n,1);
TIMES = zeros(length(rs),methodstot);
FLOPS = zeros(length(rs),methodstot);
names = [];
OUTPUTS = [];
options = MyOptions(n,m,0 ,"uni",tol,restol, maxiter,1,max_time,printiters,exacterror, "none") 
A = randn(m,n);   # randn(n,n)
(U, S, V)= svd(A);

for i =1:length(rs)
    r = rs[i]; methodcount =1;
    A= U[:, 1: r]* diagm(S[1: r])* V[:, 1: r]';
    title = string("uniform-random" , string(m) , "X" , string(n) , "_r_" , string(r))
    prob.A =A;
    prob.name = title;
#     prob = uniform_mat_rank(m,n,r);
    OUTPUTS = [];
    options.skip_error_calculation = 2;#ceil((n)^(1/3))
    
    method_name = "SATAX";#"SATAX"; "NewtonSchultz" "SAXAS"
    options.sketch = "ada";
    options.M0type = "ATproj";
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];
    names= [names; output.name];
    TIMES[i,methodcount] = output.times[end]; 
    FLOPS[i,methodcount] = output.flopsperiter*output.iterations; 
    methodcount+=1; 

    
    options.sketch = "uni";
    options.M0type = "ATproj";
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];
    names= [names; output.name];
    TIMES[i,methodcount] = output.times[end]; 
    FLOPS[i,methodcount] = output.flopsperiter*output.iterations; 
    methodcount+=1;     

    method_name = "NewtonSchulz";#"SATAX"; "NewtonSchultz" "SAXAS"
    output= pseudoinvert_matrix(prob, method_name, options);
    OUTPUTS = [OUTPUTS ; output];
    names= [names; output.name];
    TIMES[i,methodcount] = output.times[end]; 
    FLOPS[i,methodcount] = output.flopsperiter*output.iterations; 
    methodcount+=1; 
end


In [ ]:
pgfplots()
# Plotting times
Ys = TIMES;
plot(rs,Ys[:,1],label  = names[1],linestyle=:auto, marker =:auto,xlabel = "rank", ylabel = "times",  grid = false)
for i =2:methodstot
    plot!(rs,Ys[:,i],label  = names[i],linestyle=:auto, marker =:auto, grid = false)
end     
savefig("figures/$(prob.name)-rs-times.pdf");
# Plotting flops
Ys = FLOPS./maximum(FLOPS);
plot(rs,Ys[:,1],label  = names[1],linestyle=:auto, marker =:auto, xlabel = "rank", ylabel = "flops", grid = false)
for i =2:methodstot
    plot!(rs,Ys[:,i],label  = names[i],linestyle=:auto, marker =:auto, grid = false)
end

savefig("figures/$(prob.name)-rs-flops.pdf");

In [ ]:
save("./data/$(prob.name).jld", "TIMES", TIMES, "FLOPS", FLOPS, "names", names)

In [6]:
# Load saved data
probname = "uniform-random-sym-8000_r_1801"#"gisette_scale-ridge";
test = load("./data/$(probname).jld")
TIMES = test["TIMES"];
FLOPS = test["FLOPS"];
names = test["names"];